In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import glob
import numpy as np
import os
import whisper
import torch
from tqdm import tqdm
import json

In [ ]:
# Load mô hình Whisper (medium, large có thể nhận dạng tiếng Việt tốt)
model = whisper.load_model("medium")
options = whisper.DecodingOptions(language="vi", without_timestamps=True)

In [ ]:
audios_dirs = [f"{x}/Audios" for x in glob.glob('/kaggle/input/*')] #/kaggle/input/audio-123/Audios
all_audio_paths = dict()
for audios_dir in audios_dirs:
    for part in sorted(os.listdir(audios_dir)):
        all_audio_paths[part] =  dict()
for audios_dir in audios_dirs:
    for data_part in sorted(all_audio_paths.keys()):
        data_part_path = f'{audios_dir}/{data_part}'
        if os.path.isdir(data_part_path):
            audio_paths = sorted(os.listdir(data_part_path))
            for audio_path in audio_paths:
                audio_id = audio_path.replace('.wav', '')
                audio_path_full = f'{data_part_path}/{audio_path}'
                all_audio_paths[data_part][audio_id] = audio_path_full

## Inference

In [ ]:
save_dir_all = './audio'
if not os.path.exists(save_dir_all):
    os.mkdir(save_dir_all)

for key in tqdm(all_audio_paths.keys()):
    save_dir = f'{save_dir_all}/{key}'

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        
    audio_paths_dict = all_audio_paths[key]
    audio_ids = sorted(audio_paths_dict.keys())
    for audio_id in tqdm(audio_ids):
        audio_path = audio_paths_dict[audio_id]
        
        # inference
        result = model.transcribe(audio_path, language="vi", without_timestamps=True)
        sentences = result["text"].split('.')
        with open(f'/kaggle/working/audio/{key}/{audio_id}.json', 'w', encoding='utf-8') as f:
            json.dump(sentences, f, ensure_ascii=False)